In [9]:
import numpy as np
import scipy.io.wavfile
from scipy.fftpack import dct
import load_wav_files as lw

# Features Extraction

In [10]:
x_train, y_train = lw.load_dataset("speech_commands_v0.01", {'bed'}, 0.25, 'training')

/usr/lib/python3/dist-packages/scipy/io/wavfile.py:267: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [11]:
x_test, y_test = lw.load_dataset("speech_commands_v0.01", {'bed'}, 0.25, 'testing')

/usr/lib/python3/dist-packages/scipy/io/wavfile.py:267: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


### CNN

In [12]:
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

In [13]:
# Create the CNN

feat_rows = 99
feat_cols = 40
batch_size = 1716
feats_shape = (99, 40, 1)

x_train = x_train.reshape(x_train.shape[0], *feats_shape) 
x_test = x_test.reshape(x_test.shape[0], *feats_shape)
#x_validate = x_validate.reshape(x_validate.shape[0], *feats_shape)

print('x_train shape: {}'.format(x_train.shape))
#print('x_test shape: {}'.format(x_test.shape)
#print('x_validate shape: {}'.format(x_validate.shape))


x_train shape: (1716, 99, 40, 1)


In [14]:
# Create the CNN

n_kws = 31

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = 3, strides=(1, 1), input_shape = feats_shape, 
            padding='valid', data_format='channels_last', dilation_rate=(1, 1),
            activation='relu', bias_initializer='zeros', use_bias=True, kernel_initializer='glorot_uniform'))

model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3,3), strides=(1, 1),  padding='valid',
                 data_format='channels_last', dilation_rate=(1, 1),
                 activation='relu', use_bias=True, kernel_initializer='glorot_uniform', 
                 bias_initializer='zeros'))
model.add(MaxPooling2D(1))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(n_kws, activation='softmax'))


AttributeError: can't set attribute

In [ ]:
tensorboard = TensorBoard(log_dir = r'\logs{}'.format('cnn_1layer'), 
                         write_graph = True, write_grads = True, histogram_freq = 1, write_images = True)


model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.001), metrics=['accuracy'])


In [ ]:
model.fit(x_train, y_train, batch_size = batch_size, epochs=15,
          verbose = 1) 

Epoch 1/10
1716/1716 [==============================] - 115s - loss: 15.3841 - acc: 0.0437
Epoch 2/10
1716/1716 [==============================] - 88s - loss: 15.5720 - acc: 0.0344
Epoch 3/10
1716/1716 [==============================] - 107s - loss: 15.5907 - acc: 0.0332
Epoch 4/10
1716/1716 [==============================] - 27s - loss: 15.4122 - acc: 0.0443


In [ ]:
score = model.evaluate(x_test, y_test, batch_size=16, verbose = 0)

print('test loss: {:.4f}'.format(score[0]))
print('test acc: {:.4f}'.format(score[1]))